# Custom Sklearn estimators for cheminformatics: Scaffold splits example

Molpipeline provides some custom Sklearn-like estimators solving common cheminformatics tasks. The estimators comply the [Sklearn estimator API](https://scikit-learn.org/stable/developers/develop.html) and can be used in pipelines.

This notebook shows how to use the MurckoScaffoldClustering estimator for generating scaffold splits for molecular machine learning. It applies the widely used [Murcko-type decomposition](https://www.rdkit.org/docs/GettingStartedInPython.html#murcko-decomposition) to a molecule data sets. From the decomposition a clustering is generated which then can be directly used with Sklearn's group-based splitters for cluster cross-validation.

This is a simple example notebook using dummy data to illustrate the usage of custom estimators for cheminformatics like MurckoScaffoldClustering. Please look at the advanced notebooks for more detailed examples. 

[**Scaffold clustering**](#estimators)
* Murcko-scaffolds and generic scaffolds with MurckoScaffoldClustering estimator

[**Putting it together**](#fullexample)
* Train and evaluate a classfier with MolPipeline
* Cross validation evaluation with scaffold-split
    * Combine MurckoScaffoldClustering with Sklearn's GroupKFold

In [1]:
import numpy as np

from molpipeline import Pipeline
from molpipeline.any2mol import AutoToMol
from molpipeline.mol2mol import ElementFilter
from molpipeline.mol2any import MolToMorganFP
from molpipeline.estimators import MurckoScaffoldClustering

import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


## Scaffold clustering <a class="anchor" id="estimators"></a>

MolPipeline implements custom Sklearn estimators for standard molecular machine learning tasks. For example, we created a MurckoScaffoldClustering estimator which can be used like a normal Sklearn clustering estimator.   

In [2]:
scaffold_smiles = [
    "Nc1ccccc1",
    "Cc1cc(Oc2nccc(CCC)c2)ccc1",
    "c1ccccc1",
]
linear_smiles = ["CC", "CCC", "CCCN"]

# run the scaffold clustering
scaffold_clustering = MurckoScaffoldClustering(
    n_jobs=1, linear_molecules_strategy="ignore"
)
scaffold_clustering.fit_predict(scaffold_smiles + linear_smiles)

array([ 1.,  0.,  1., nan, nan, nan])

The cluster labels of the above scaffold clustering assigns nan to linear molecules. This is because we used linear_molecules_strategy="ignore". Instead we can also use the "own_cluster" strategy which groups all linear molecules in a new cluster:

In [3]:
scaffold_clustering = MurckoScaffoldClustering(
    n_jobs=1, linear_molecules_strategy="own_cluster"
)
scaffold_clustering.fit_predict(scaffold_smiles + linear_smiles)

array([1., 0., 1., 2., 2., 2.])

In addition, instead of using the basic Murcko scaffolds for clustering we can cluster with the generic scaffolds. Generic scaffolds are Murcko scaffolds but with all atom elements set to carbons. We can do this by setting make_generic=True like this:

In [4]:
scaffold_clustering = MurckoScaffoldClustering(
    n_jobs=1, linear_molecules_strategy="own_cluster", make_generic=True
)
scaffold_clustering.fit_predict(scaffold_smiles + linear_smiles + ["c1ncccc1"])

array([1., 0., 1., 2., 2., 2., 1.])

## Cross validation with scaffold split <a class="anchor" id="fullexample"></a>

A cross validation with scaffold splits is straightfoward to implement with MurckoScaffoldClustering. We can simply combine the generated scaffold clusters with Sklearn's group-based splitters, like [GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html), [StratifiedGroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedGroupKFold.html), [LeaveOneGroupOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut), etc.

Let's setup some data and do the clustering

In [5]:
# a list of dummy smiles
smiles_data = np.array(
    [
        "Nc1ccccc1",
        "Cc1cc(Oc2nccc(CCC)c2)ccc1",
        "c1ccccc1",
        "CCCCN",
        "CCC",
        "CCO",
        "Oc1ccccc1",
        "Oc1ccc(N)cc1",
    ]
)
# a simple dummy target variable y that indicates whether the molecule contains a nitrogen (1=has N, 0=no N)
has_nitrogen_label = np.array([1, 1, 0, 1, 0, 0, 0, 1])

# we cluster the molecules by their murcko scaffold for our cross validation split
scaffold_clustering = MurckoScaffoldClustering(
    n_jobs=1, linear_molecules_strategy="own_cluster"
)
groups = scaffold_clustering.fit_predict(smiles_data)

# let's look at the data in a nice dataframe
df = pd.DataFrame(
    {
        "smiles": smiles_data,
        "has_nitrogen_label": has_nitrogen_label,
        "murcko_clusters": groups.astype(int),
    }
)
PandasTools.AddMoleculeColumnToFrame(df, "smiles", "Molecule")
PandasTools.AddMurckoToFrame(df, molCol="Molecule")
df.set_index("smiles")

Failed to patch pandas - unable to change molecule rendering


,has_nitrogen_label,murcko_clusters,Molecule,Murcko_SMILES
smiles,,,,
Nc1ccccc1,1,1,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf40580>,c1ccccc1
Cc1cc(Oc2nccc(CCC)c2)ccc1,1,0,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf403c0>,c1ccc(Oc2ccccn2)cc1
c1ccccc1,0,1,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf40510>,c1ccccc1
CCCCN,1,2,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf405f0>,
CCC,0,2,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf40660>,
CCO,0,2,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf409e0>,
Oc1ccccc1,0,1,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf40a50>,c1ccccc1
Oc1ccc(N)cc1,1,1,<rdkit.Chem.rdchem.Mol object at 0x7f8bdbf40ac0>,c1ccccc1


Now we can run a cross validation using GroupKFold

In [6]:
# setup a splitter that handles the cluster split for us
grouper = GroupKFold(n_splits=2)
grouper.random_state = 67056

for i, (train, test) in enumerate(
    grouper.split(smiles_data, has_nitrogen_label, groups=groups)
):

    # setup the pipeline
    pipeline = Pipeline(
        [("auto2mol", AutoToMol())]
        + [
            (
                "morgan",
                MolToMorganFP(n_bits=1024, radius=2),
            ),
            ("RandomForestClassifier", RandomForestClassifier(random_state=67056)),
        ]
    )

    # fit the pipeline to the training data
    pipeline.fit(X=smiles_data[train], y=has_nitrogen_label[train])

    # evaluate the pipeline on the test set
    predictions = pipeline.predict_proba(
        X=smiles_data[test],
    )

    # print the performance for predicting the presence of nitrogens on the test set
    for smi, pred, label in zip(
        smiles_data[test], predictions[:, 1], has_nitrogen_label[test]
    ):
        print(f"fold {i}:", smi, f"prediction={pred:.2f}", f"label={label}")
    print(
        f"fold {i}:",
        "test ROC AUC score:",
        roc_auc_score(has_nitrogen_label[test], predictions[:, 1]),
    )
    print("-------------------")

fold 0: Nc1ccccc1 prediction=0.31 label=1
fold 0: c1ccccc1 prediction=0.20 label=0
fold 0: Oc1ccccc1 prediction=0.22 label=0
fold 0: Oc1ccc(N)cc1 prediction=0.27 label=1
fold 0: test ROC AUC score: 1.0
-------------------
fold 1: Cc1cc(Oc2nccc(CCC)c2)ccc1 prediction=0.29 label=1
fold 1: CCCCN prediction=0.30 label=1
fold 1: CCC prediction=0.24 label=0
fold 1: CCO prediction=0.27 label=0
fold 1: test ROC AUC score: 1.0
-------------------


The results above show that in this dummy example the presence of a nitrogen can be learned and predicted with a ROC AUC score of 1.0 on the test sets of the scaffold split.   